# Download Dataset

In [1]:
!kaggle datasets download -d laotse/credit-risk-dataset

  0%|                                                | 0.00/368k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 368k/368k [00:00<00:00, 5.10MB/s]


In [2]:
!unzip credit-risk-dataset.zip

Archive:  credit-risk-dataset.zip
  inflating: credit_risk_dataset.csv  


In [3]:
!rm credit-risk-dataset.zip

# Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io

io.templates.default = "plotly_dark"
io.renderers.default = "notebook_connected"


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import optuna

# EDA

In [5]:
df = pd.read_csv("credit_risk_dataset.csv")
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [6]:
df.describe()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.000000,3.258100e+04,31686.000000,32581.000000,29465.000000,32581.000000,32581.000000,32581.000000
mean,27.734600,6.607485e+04,4.789686,9589.371106,11.011695,0.218164,0.170203,5.804211
std,6.348078,6.198312e+04,4.142630,6322.086646,3.240459,0.413006,0.106782,4.055001
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,7.920000e+04,7.000000,12200.000000,13.470000,0.000000,0.230000,8.000000
max,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [17]:
df.dtypes

person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
dtype: object

In [37]:
df.fillna(df.mean(), inplace=True)

/tmp/ipykernel_26136/820435583.py:1: FutureWarning:

The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



In [38]:
df.isnull().sum()

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

In [39]:
str_columns = df.dtypes[df.dtypes == "object"].index
str_columns

Index(['person_home_ownership', 'loan_intent', 'loan_grade',
       'cb_person_default_on_file'],
      dtype='object')

In [40]:
for col in str_columns:
    df[col] = df[col].apply(lambda x: str(x).strip())

In [41]:
df.dtypes

person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_status                     int64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
dtype: object

In [42]:
df["loan_status"].value_counts()/len(df)

0    0.781836
1    0.218164
Name: loan_status, dtype: float64

In [48]:
X = df.drop("loan_status", axis=1)
y = df["loan_status"]
cat_features = X.columns[np.where(X.dtypes != float)[0]].values.tolist()

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
catboost = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, loss_function="Logloss", eval_metric="AUC", random_seed=42, use_best_model=True)
catboost.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100, cat_features=cat_features)

0:	test: 0.8452308	best: 0.8452308 (0)	total: 107ms	remaining: 1m 46s
100:	test: 0.9570248	best: 0.9570248 (100)	total: 5.74s	remaining: 51s
200:	test: 0.9613269	best: 0.9613370 (193)	total: 12s	remaining: 47.7s
300:	test: 0.9625046	best: 0.9625046 (300)	total: 19s	remaining: 44.1s
400:	test: 0.9629026	best: 0.9630238 (363)	total: 24.8s	remaining: 37s
500:	test: 0.9636308	best: 0.9636308 (500)	total: 30.6s	remaining: 30.5s
600:	test: 0.9633975	best: 0.9636308 (500)	total: 36.7s	remaining: 24.4s
700:	test: 0.9631335	best: 0.9636308 (500)	total: 43.3s	remaining: 18.5s
800:	test: 0.9633454	best: 0.9636308 (500)	total: 50.4s	remaining: 12.5s
900:	test: 0.9629589	best: 0.9636308 (500)	total: 56.9s	remaining: 6.25s
999:	test: 0.9630222	best: 0.9636308 (500)	total: 1m 3s	remaining: 0us

bestTest = 0.9636308439
bestIteration = 500

Shrink model to first 501 iterations.


In [51]:
catboost.get_best_score()

{'learn': {'Logloss': 0.11028521931653382},
 'validation': {'Logloss': 0.16496361740286555, 'AUC': 0.9636308438758692}}

In [52]:
y_pred = catboost.predict(X_test)

In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5072
           1       0.96      0.78      0.86      1445

    accuracy                           0.94      6517
   macro avg       0.95      0.88      0.91      6517
weighted avg       0.94      0.94      0.94      6517



In [54]:
accuracy_score(y_test, y_pred)

0.9427650759551941

In [57]:
#Fine tune using optuna

def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'depth': trial.suggest_int('depth', 3, 10),
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'random_seed': 42,
        'use_best_model': True,
        'verbose': 100
    }
    catboost = CatBoostClassifier(**params)
    catboost.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=cat_features)
    return catboost.get_best_score()['validation']['AUC']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-01-11 12:07:12,623] A new study created in memory with name: no-name-45b2860b-2492-46ec-85de-fc9f3d571bfb
/tmp/ipykernel_26136/780873226.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



0:	test: 0.8427404	best: 0.8427404 (0)	total: 69.6ms	remaining: 1m 9s
100:	test: 0.8630100	best: 0.8632258 (91)	total: 3.84s	remaining: 34.2s
200:	test: 0.9161063	best: 0.9161063 (200)	total: 7.24s	remaining: 28.8s
300:	test: 0.9312711	best: 0.9312711 (300)	total: 10.2s	remaining: 23.7s
400:	test: 0.9370707	best: 0.9370707 (400)	total: 13.3s	remaining: 19.8s
500:	test: 0.9399149	best: 0.9399149 (500)	total: 16.6s	remaining: 16.6s
600:	test: 0.9418791	best: 0.9418791 (600)	total: 20.1s	remaining: 13.3s
700:	test: 0.9433868	best: 0.9433876 (699)	total: 23.8s	remaining: 10.1s
800:	test: 0.9449891	best: 0.9449891 (800)	total: 27.3s	remaining: 6.79s
900:	test: 0.9464747	best: 0.9464747 (900)	total: 31.2s	remaining: 3.42s


[I 2023-01-11 12:07:48,702] Trial 0 finished with value: 0.9483293855675504 and parameters: {'learning_rate': 0.004941949914593827, 'depth': 4}. Best is trial 0 with value: 0.9483293855675504.


999:	test: 0.9483294	best: 0.9483294 (999)	total: 35.2s	remaining: 0us

bestTest = 0.9483293856
bestIteration = 999

0:	test: 0.8375186	best: 0.8375186 (0)	total: 38.8ms	remaining: 38.8s
100:	test: 0.8450211	best: 0.8450218 (88)	total: 2.89s	remaining: 25.7s
200:	test: 0.8480512	best: 0.8480731 (197)	total: 6.33s	remaining: 25.2s
300:	test: 0.8579926	best: 0.8580121 (285)	total: 9.19s	remaining: 21.3s
400:	test: 0.8629460	best: 0.8633868 (399)	total: 12s	remaining: 17.9s
500:	test: 0.8631463	best: 0.8633868 (399)	total: 14.9s	remaining: 14.8s
600:	test: 0.8635892	best: 0.8636635 (525)	total: 17.7s	remaining: 11.7s
700:	test: 0.8962782	best: 0.8962782 (700)	total: 20.2s	remaining: 8.62s
800:	test: 0.9094312	best: 0.9094312 (800)	total: 22.8s	remaining: 5.67s
900:	test: 0.9144484	best: 0.9144484 (900)	total: 25.5s	remaining: 2.8s


[I 2023-01-11 12:08:17,681] Trial 1 finished with value: 0.919175771997424 and parameters: {'learning_rate': 0.0012737888340198951, 'depth': 3}. Best is trial 0 with value: 0.9483293855675504.


...

999:	test: 0.9629961	best: 0.9630110 (953)	total: 1m 41s	remaining: 0us

bestTest = 0.963010981
bestIteration = 953

Shrink model to first 954 iterations.
0:	test: 0.8534399	best: 0.8534399 (0)	total: 97ms	remaining: 1m 36s
100:	test: 0.9575517	best: 0.9575751 (95)	total: 10.8s	remaining: 1m 35s
200:	test: 0.9606224	best: 0.9608819 (183)	total: 21.8s	remaining: 1m 26s
300:	test: 0.9601762	best: 0.9608819 (183)	total: 32s	remaining: 1m 14s
400:	test: 0.9602706	best: 0.9608819 (183)	total: 42.3s	remaining: 1m 3s
500:	test: 0.9605924	best: 0.9608819 (183)	total: 52.7s	remaining: 52.4s
600:	test: 0.9603030	best: 0.9608819 (183)	total: 1m 2s	remaining: 41.8s
700:	test: 0.9602978	best: 0.9608819 (183)	total: 1m 13s	remaining: 31.2s
800:	test: 0.9606792	best: 0.9608819 (183)	total: 1m 23s	remaining: 20.7s
900:	test: 0.9603971	best: 0.9608819 (183)	total: 1m 34s	remaining: 10.3s


[I 2023-01-11 12:18:55,767] Trial 10 finished with value: 0.9608819163219193 and parameters: {'learning_rate': 0.09823668121981591, 'depth': 10}. Best is trial 6 with value: 0.963189721982688.


999:	test: 0.9604606	best: 0.9608819 (183)	total: 1m 44s	remaining: 0us

bestTest = 0.9608819163
bestIteration = 183

Shrink model to first 184 iterations.
0:	test: 0.8534399	best: 0.8534399 (0)	total: 103ms	remaining: 1m 42s
100:	test: 0.9551600	best: 0.9551600 (100)	total: 9.44s	remaining: 1m 24s
200:	test: 0.9592072	best: 0.9592072 (200)	total: 19.8s	remaining: 1m 18s
300:	test: 0.9613217	best: 0.9613217 (300)	total: 31.8s	remaining: 1m 13s


[W 2023-01-11 12:19:30,684] Trial 11 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "/home/hari31416/anaconda3/envs/data-science/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_26136/780873226.py", line 15, in objective
    catboost.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=cat_features)
  File "/home/hari31416/anaconda3/envs/data-science/lib/python3.9/site-packages/catboost/core.py", line 5128, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/home/hari31416/anaconda3/envs/data-science/lib/python3.9/site-packages/catboost/core.py", line 2355, in _fit
    self._train(
  File "/home/hari31416/anaconda3/envs/data-science/lib/python3.9/site-packages/catboost/core.py", line 1759, in _train
    self._object._train(train_pool

KeyboardInterrupt: 

In [60]:
best_params = study.best_params

In [61]:
final_model = CatBoostClassifier(iterations=1000, learning_rate=best_params["learning_rate"], depth=best_params["depth"], loss_function="Logloss", eval_metric="AUC", random_seed=42, use_best_model=True)

In [62]:
final_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100, cat_features=cat_features)

0:	test: 0.8534399	best: 0.8534399 (0)	total: 109ms	remaining: 1m 48s
100:	test: 0.9564442	best: 0.9564442 (100)	total: 9.6s	remaining: 1m 25s
200:	test: 0.9609780	best: 0.9609780 (200)	total: 20s	remaining: 1m 19s
300:	test: 0.9620657	best: 0.9620978 (294)	total: 32.2s	remaining: 1m 14s
400:	test: 0.9626681	best: 0.9627127 (384)	total: 43.4s	remaining: 1m 4s
500:	test: 0.9629969	best: 0.9630757 (494)	total: 54.7s	remaining: 54.4s
600:	test: 0.9631411	best: 0.9631897 (567)	total: 1m 5s	remaining: 43.6s
700:	test: 0.9628981	best: 0.9631897 (567)	total: 1m 17s	remaining: 32.9s
800:	test: 0.9628661	best: 0.9631897 (567)	total: 1m 29s	remaining: 22.2s
900:	test: 0.9627743	best: 0.9631897 (567)	total: 1m 41s	remaining: 11.2s
999:	test: 0.9628268	best: 0.9631897 (567)	total: 1m 53s	remaining: 0us

bestTest = 0.963189722
bestIteration = 567

Shrink model to first 568 iterations.


In [63]:
y_pred = final_model.predict(X_test)
print(classification_report(y_test, y_pred))
accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.94      0.99      0.96      5072
           1       0.96      0.76      0.85      1445

    accuracy                           0.94      6517
   macro avg       0.95      0.88      0.91      6517
weighted avg       0.94      0.94      0.94      6517



0.9398496240601504